In [14]:
import torch
from torch import nn
from torch.distributions.multivariate_normal import MultivariateNormal
import numpy as np
import math
from torch.utils.data import Dataset, DataLoader
import pdb 

In [22]:
def preprocess(path):
    file_data = []
    with open(path, 'r') as file:
        for line in file:
            line_data = [int(float(data)) if i < 2 else float(data) for i, data in enumerate(line.rsplit())]
            line_data[2], line_data[3] = line_data[3], line_data[2]
            file_data.append(line_data)
    file_data.sort(key=lambda data : data[0])

    file_data_t = []
    data_temp = []
    frame_num = file_data[0][0]
    traj_list = []
    frame_list = []
    for line in file_data:
        if frame_num != line[0]:
            frame_num = line[0]
            data_temp.sort(key=lambda data : data[1])
            file_data_t.append(data_temp)
            data_temp = [line]
        else:    
            data_temp.append(line)
        if line[1] not in traj_list:
            traj_list.append(line[1])
        if line[0] not in frame_list:
            frame_list.append(line[0])
        
    traj_list.sort()
    frame_list.sort()

    #get participants in each frame
    #@note here the elements are ped's index in the traj list
    participants = [[] for i in range(len(file_data_t))]
    for frame_idx, line in enumerate(file_data_t):
        for traj_idx, traj in enumerate(traj_list):
            in_flag = False
            for data in line:
                if data[1] == traj:
                    in_flag = True
                    participants[frame_idx].append(traj_list.index(data[1]))
            if not in_flag:
                file_data_t[frame_idx].append([frame_list[frame_idx], traj, 0., 0.])
        file_data_t[frame_idx].sort(key=lambda data : data[1])

    file_data_tensors = torch.tensor(file_data_t)

    participant_masks = []
    for frame_idx, line in enumerate(participants):
        participant_masks.append([[torch.tensor(1.) if i in participants[frame_idx] else torch.tensor(0.) for i in range(len(traj_list)) ]])
    participant_masks = torch.tensor(participant_masks)

    # for line in participant_masks:
    #     print(line, line.shape)

    # print(file_data_tensors.shape)
    # for t, line in enumerate(file_data_tensors):
    #     if t <= 20:
    #         print(line,'\n')
    #         print(participants[t])
    

    return traj_list, participant_masks, file_data_tensors


class FramesDataset(Dataset):
    def __init__(self, path):
        self.traj_list, self.participant_masks, self.file_data = preprocess(path)

    def __len__(self):
        return len(self.file_data)

    def __getitem__(self, idx):
        if isinstance(idx, int):
            if idx < len(self.file_data)-1:
                Y_idx = idx+1
            else:
                Y_idx = len(self.file_data)-1

        else:
            if idx.start != None:
                start = idx.start+1
            else:
                start = 0+1
            if idx.stop != None:
                stop = idx.stop+1
            else:
                stop = len(self.file_data)-1
            Y_idx = slice(start, stop)

        participant_mask = self.participant_masks[idx]
        X = self.file_data[idx]
        Y = self.file_data[Y_idx]

        return participant_mask, (X, Y)

    def getTrajList(self):
        return self.traj_list

    def getParticipants(self):
        return self.participants


D = FramesDataset("biwi_hotel_4.txt")
Loader = DataLoader(D, batch_size=20)

for part, (X, Y) in Loader:
    print(part.shape, X.shape, Y.shape)
# print(data[:4])
# for line in data:
#     print(line,'\n')
# for (X, Y) in Loader:
#     print(X.shape)
#     for linex, liney in zip(X,Y):
#         print('X')
#         print(linex[:4], '\nY\n', liney[:4])
#         print('===================================================')

torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) torch.Size([20, 145, 4]) torch.Size([20, 145, 4])
torch.Size([20, 1, 145]) 

In [16]:
class VanillaLSTM(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=20, mediate_dim=10, output_dim=2, traj_num=3):
        super(VanillaLSTM, self).__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.traj_num = traj_num
        self.InputLayer = nn.Linear(input_dim, mediate_dim)
        self.LSTMCell = nn.LSTMCell(mediate_dim, hidden_dim)
        self.OutputLayer = nn.Linear(hidden_dim, output_dim)


    def forward(self, X, part_masks, all_h_t, all_c_t, T_obs, T_pred):
        outputs = torch.empty(X.shape[0], X.shape[1], self.output_dim)
        for frame_idx, x in enumerate(X):      
            if frame_idx <= T_obs or frame_idx > T_pred:
                outputs[frame_idx] = torch.zeros(X.shape[1], self.output_dim)
                continue

            r = self.InputLayer(x[:,2:])
            all_h_t, all_c_t = self.LSTMCell(r, (all_h_t, all_c_t))
            part_mask = torch.t(part_masks[frame_idx]).expand(part_masks[frame_idx].shape[1], self.output_dim)
            outputs[frame_idx] = self.OutputLayer(all_h_t) * part_mask

        return outputs

In [23]:
def train(T_obs, T_pred):
    h_dim = 100

    #try to train this
    dataset = FramesDataset("biwi_hotel_4.txt")
    #a dataloader for now not sure how to use
    dataloader = DataLoader(dataset, batch_size=24)

    part_masks, (input_seq, Y) = dataset[:21]
    Y = Y[:,:,2:]
    traj_num = len(dataset.getTrajList())
    h = torch.zeros(traj_num, h_dim)
    c = torch.zeros(traj_num, h_dim)

    vl = VanillaLSTM(hidden_dim=h_dim, mediate_dim=10, output_dim=2, traj_num=traj_num)

    #define loss & optimizer
    criterion = nn.MSELoss(reduction="sum")
    # criterion = Gaussian2DNll
    
    optimizer = torch.optim.Adagrad(vl.parameters(), weight_decay=0.0005)

    for epoch in range(10):
        with torch.autograd.set_detect_anomaly(True):         
            #forward prop
            output = vl(input_seq, part_masks, h, c, T_obs, T_pred)

            #compute loss
            Y_pred = output[T_obs+1:T_pred]
            Y_g = Y[T_obs+1:T_pred]
            cost = criterion(Y_pred, Y_g)

            if epoch % 10 == 9:
                print(epoch, cost.item())

            #backward prop
            optimizer.zero_grad()
            cost.backward()
            optimizer.step()
    return vl


def validate(model, T_obs, T_pred):
    #try to validate this
    h_dim = 100

    dataset = FramesDataset("biwi_hotel_4.txt")
    #a dataloader for now not sure how to use
    dataloader = DataLoader(dataset, batch_size=24)

    part_masks, (input_seq, Y) = dataset[40:61]
    Y = Y[:,:,2:]
    traj_num = len(dataset.getTrajList())
    h = torch.zeros(traj_num, h_dim)
    c = torch.zeros(traj_num, h_dim)
    
    with torch.autograd.set_detect_anomaly(True):         
        output = model(input_seq, part_masks, h, c, T_obs, T_pred)

        #compute loss
        Y_pred = output[T_obs+1:T_pred]
        Y_g = Y[T_obs+1:T_pred]

        print("\nvalidating")
        for frame_idx, (y_pred, y_g) in enumerate(zip(Y_pred, Y_g)):
            for traj_idx in range(part_masks[frame_idx].shape[1]):
                if part_masks[frame_idx][0][traj_idx] != 0:
                    print("at frame", frame_idx+T_obs+1, "ped", traj_idx, "is off by", torch.dist(y_pred[traj_idx],y_g[traj_idx]).item())
            print("")

if __name__ == "__main__":
    vl = train(12, 20)
    # validate(vl, 12, 20)

> <ipython-input-16-f4ac5afd3cd5>(16)forward()
-> if frame_idx <= T_obs or frame_idx > T_pred:


BdbQuit: 

In [18]:
import torch
import numpy as np
from torch.distributions.multivariate_normal import MultivariateNormal

def Gaussian2Dmy(params, event):
    y = event.view(2,-1)

    mu_x = params[0].view(1,1)
    mu_y = params[1].view(1,1)
    sig_x = params[2].view(1,1)
    sig_y = params[3].view(1,1)
    rho_xy = params[4].view(1,1)

    mu = torch.cat((mu_x,mu_y))

    covariance = rho_xy*sig_x*sig_y
    covariance = covariance.view(1,1)

    cov_mat_r1 = torch.cat((sig_x**2, covariance), 1)
    cov_mat_r2 = torch.cat((covariance, sig_y**2), 1)
    cov_mat = torch.cat((cov_mat_r1, cov_mat_r2))

    covmat_det = torch.sum((sig_x**2*sig_y**2)) - covariance**2

    things_inexp = -1/2*torch.mm( torch.t(y-mu),torch.mm(torch.inverse(cov_mat),(y-mu)) )

    coeff_before_exp = 1/(2*np.pi*torch.sqrt(covmat_det)) 

    P = coeff_before_exp * torch.exp(things_inexp)
    logP = torch.sum(torch.log(P))

    return logP

params = torch.tensor([-0.1191,  0.5695,  0.1431,  0.0246,  0.5175])
mu = params[:2]
(sig_x, sig_y) = (params[2], params[3])
cov = sig_x*sig_y*params[4]
covmat = torch.tensor([[sig_x**2, cov],[cov, sig_y**2]])
y = torch.tensor([1.7,-0.7])

rv = MultivariateNormal(mu, covmat)
print(rv.log_prob(y))
print(Gaussian2Dmy(params, y))

tensor(-2388.6519)
tensor(-inf)


In [19]:
# import torch
# from torch import nn

# class TwoParts(nn.Module):
#     def __init__(self):
#         super(TwoParts, self).__init__()
#         self.lin = nn.Linear(20, 5)
#         self.rnn = nn.ModuleList([nn.LSTMCell(10, 20) for _ in range(10)])
#         self.lin1 = nn.ModuleList([nn.Linear(10, 10) for _ in range(10)])

#     def forward(self, input, hx, cx):
#         output = torch.zeros(6, 3, 5)
#         h = [torch.zeros(3, 20) for _ in range(10)]
#         c = [torch.zeros(3, 20) for _ in range(10)]
#         h[0] = hx
#         c[0] = cx
#         for i in range(6):
#             x = self.lin1[0](input[i])
#             x = x.view(3, -1)
#             if i == 1:
#                 h[0], c[0] =  0*h[0], 0*c[0]
#                 continue
#             h[0], c[0] = self.rnn[0](x, (h[0], c[0]))
#             output[i] = self.lin(h[0])

#         return output

# torch.manual_seed(0)
# input = torch.randn(6, 3, 10)
# Y = torch.randn(6, 3, 5)
# hx = torch.randn(3, 20)
# cx = torch.randn(3, 20)

# model = TwoParts()
# criterion = torch.nn.MSELoss(reduction='sum')
# optimizer = torch.optim.Adam(model.parameters())
# for j in range(30):
#     output = model(input, hx, cx)

#     loss = criterion(output, Y)

#     print(loss.item())

#     optimizer.zero_grad()
#     if j == 0:
#         loss.backward(retain_graph=True)
#     else:
#         loss.backward()
#     optimizer.step()
    

# class C():
#     def __init__(self, data):
#         self.data = data

#     def __getitem__(self,val):
#         print(val.stop)
#         start = val.start+1
#         stop = val.stop+1
#         Y_slice = slice(start, stop)

#         X = self.data[val]
#         Y = self.data[Y_slice]

#         return X, Y

# c = C([0,1,2,3,4,5])
# x, y = c[2:5]
# print(x, y)

x = tensor

NameError: name 'tensor' is not defined